In [43]:
import ee

In [44]:
ee.Authenticate()

Enter verification code: 4/1AdQt8qgZhyHAYkRBkRUGDte3VPtw4U_sTIR5LhBdmZMs8SIIXnYRsEzARQ0

Successfully saved authorization token.


In [45]:
ee.Initialize()

In [46]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import boto3
import pandas as pd

# Load data

In [47]:
######### World cover

##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection()

In [48]:
######## LANDSAT

COLLECTION = ee.Dictionary({
  'L4': {
    'TOA': ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT04/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L5': {
    'TOA': ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT05/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L7': {
    'TOA': ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LE07/C01/T1_SR'),
    'TIR': ['B6_VCID_1','B6_VCID_2'],
  },
  'L8': {
    'TOA': ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LC08/C01/T1_SR'),
    'TIR': ['B10','B11']
  }
})

In [49]:
########### Aester emissivity

# get ASTER emissivity
aster = ee.Image("NASA/ASTER_GED/AG100_003")

#get ASTER FVC from NDVI
aster_ndvi = aster.select('ndvi').multiply(0.01)

aster_fvc = aster_ndvi.expression('((ndvi-ndvi_bg)/(ndvi_vg - ndvi_bg))**2',
  {'ndvi':aster_ndvi,'ndvi_bg':0.2,'ndvi_vg':0.86})
aster_fvc = aster_fvc.where(aster_fvc.lt(0.0),0.0)
aster_fvc = aster_fvc.where(aster_fvc.gt(1.0),1.0)

# Define functions

In [51]:
# add date as a band to image collection
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [52]:
def get_hottest_period(Districts,start_date,end_date,ROIcenter):
    # select dataset, filter by dates and visualize
    dataset = (ee.ImageCollection('NASA/NEX-GDDP')
               .filter(ee.Filter.And(
                   ee.Filter.date(start_date, end_date),
                   ee.Filter.eq('scenario','rcp85'),
                    ee.Filter.eq('model','BNU-ESM'),
                   ee.Filter.bounds(Districts)
               )
                      )
              )
    AirTemperature = dataset.select(['tasmax'])
    
    withdates = AirTemperature.map(addDate)
    
    # create a composite with the hottest day value and dates for every location and add to map
    hottest = withdates.qualityMosaic('tasmax')
    
    # reduce composite to get the hottest date for centroid of ROI
    resolution = dataset.first().projection().nominalScale()
    NEXtempMax = ee.Number(hottest.reduceRegion(ee.Reducer.firstNonNull(), ROIcenter, resolution).get('date'))
    
    # convert date number to date type
    date = ee.Date.parse('YYYYMMdd',str(NEXtempMax.getInfo()))
    
    # calculate 45 days before and after hottest date.  Format as short date.
    start90days = date.advance(-44, 'day').format('YYYY-MM-dd')
    end90days = date.advance(45, 'day').format('YYYY-MM-dd')
    print(start90days.getInfo())
    print(end90days.getInfo())
    start90days_info = start90days.getInfo()
    end90days_info = end90days.getInfo()
    
    return(start90days_info,end90days_info)

In [53]:
def NDVIaddBand(landsat):
  def wrap(image):

    # choose bands
    nir = ee.String(ee.Algorithms.If(landsat == 'L8','B5','B4'))
    red = ee.String(ee.Algorithms.If(landsat == 'L8','B4','B3'))

    # compute NDVI
    return image.addBands(image.expression('(nir-red)/(nir+red)',{
      'nir':image.select(nir).multiply(0.0001),
      'red':image.select(red).multiply(0.0001)
    }).rename('NDVI'))

  return wrap

In [54]:
def FVCaddBand(landsat):
  def wrap(image):

    ndvi = image.select('NDVI')

    # Compute FVC
    fvc = image.expression('((ndvi-ndvi_bg)/(ndvi_vg - ndvi_bg))**2',
      {'ndvi':ndvi,'ndvi_bg':0.2,'ndvi_vg':0.86})
    fvc = fvc.where(fvc.lt(0.0),0.0)
    fvc = fvc.where(fvc.gt(1.0),1.0)

    return image.addBands(fvc.rename('FVC'))

  return wrap

In [55]:
def NCEP_TPWaddBand(image):

  # first select the day of interest
  date = ee.Date(image.get('system:time_start'))
  year = ee.Number.parse(date.format('yyyy'))
  month = ee.Number.parse(date.format('MM'))
  day = ee.Number.parse(date.format('dd'))
  date1 = ee.Date.fromYMD(year,month,day)
  date2 = date1.advance(1,'days')

  # function compute the time difference from landsat image
  def datedist(image):
    return image.set('DateDist',
      ee.Number(image.get('system:time_start')) \
      .subtract(date.millis()).abs())
  

  # load atmospheric data collection
  TPWcollection = ee.ImageCollection('NCEP_RE/surface_wv') \
                  .filter(ee.Filter.date(date1.format('yyyy-MM-dd'), date2.format('yyyy-MM-dd'))) \
                  .map(datedist)

  # select the two closest model times
  closest = (TPWcollection.sort('DateDist')).toList(2)

  # check if there is atmospheric data in the wanted day
  # if not creates a TPW image with non-realistic values
  # these are then masked in the SMWalgorithm function (prevents errors)
  tpw1 = ee.Image(ee.Algorithms.If(closest.size().eq(0), ee.Image.constant(-999.0),
                      ee.Image(closest.get(0)).select('pr_wtr') ))
  tpw2 = ee.Image(ee.Algorithms.If(closest.size().eq(0), ee.Image.constant(-999.0),
                        ee.Algorithms.If(closest.size().eq(1), tpw1,
                        ee.Image(closest.get(1)).select('pr_wtr') )))

  time1 = ee.Number(ee.Algorithms.If(closest.size().eq(0), 1.0,
                        ee.Number(tpw1.get('DateDist')).divide(ee.Number(21600000)) ))
  time2 = ee.Number(ee.Algorithms.If(closest.size().lt(2), 0.0,
                        ee.Number(tpw2.get('DateDist')).divide(ee.Number(21600000)) ))

  tpw = tpw1.expression('tpw1*time2+tpw2*time1',
                            {'tpw1':tpw1,
                            'time1':time1,
                            'tpw2':tpw2,
                            'time2':time2
                            }).clip(image.geometry())

  # SMW coefficients are binned by TPW values
  # find the bin of each TPW value
  pos = tpw.expression(
    "value = (TPW>0 && TPW<=6) ? 0" + \
    ": (TPW>6 && TPW<=12) ? 1" + \
    ": (TPW>12 && TPW<=18) ? 2" + \
    ": (TPW>18 && TPW<=24) ? 3" + \
    ": (TPW>24 && TPW<=30) ? 4" + \
    ": (TPW>30 && TPW<=36) ? 5" + \
    ": (TPW>36 && TPW<=42) ? 6" + \
    ": (TPW>42 && TPW<=48) ? 7" + \
    ": (TPW>48 && TPW<=54) ? 8" + \
    ": (TPW>54) ? 9" + \
    ": 0",{'TPW': tpw}) \
    .clip(image.geometry())

  # add tpw to image as a band
  withTPW = (image.addBands(tpw.rename('TPW'),['TPW'])).addBands(pos.rename('TPWpos'),['TPWpos'])

  return withTPW

In [56]:
# bare ground emissivity functions for each band
def ASTERGEDemiss_bare_band10(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band10').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())

def ASTERGEDemiss_bare_band11(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band11').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())

def ASTERGEDemiss_bare_band12(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band12').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())

def ASTERGEDemiss_bare_band13(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band13').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())

def ASTERGEDemiss_bare_band14(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band14').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())

In [57]:
def EMaddBand(landsat, use_ndvi):
  def wrap(image):

    c13 = ee.Number(ee.Algorithms.If(landsat == 'L4',0.3222,
                            ee.Algorithms.If(landsat == 'L5',-0.0723,
                            ee.Algorithms.If(landsat == 'L7',0.2147,
                            0.6820))))
    c14 = ee.Number(ee.Algorithms.If(landsat == 'L4',0.6498,
                            ee.Algorithms.If(landsat == 'L5',1.0521,
                            ee.Algorithms.If(landsat == 'L7',0.7789,
                            0.2578))))
    c = ee.Number(ee.Algorithms.If(landsat == 'L4',0.0272,
                            ee.Algorithms.If(landsat == 'L5',0.0195,
                            ee.Algorithms.If(landsat == 'L7',0.0059,
                            0.0584))))

    # get ASTER emissivity
    # convolve to Landsat band
    emiss_bare = image.expression('c13*EM13 + c14*EM14 + c',{
      'EM13':ASTERGEDemiss_bare_band13(image),
      'EM14':ASTERGEDemiss_bare_band14(image),
      'c13':ee.Image(c13),
      'c14':ee.Image(c14),
      'c':ee.Image(c)
      })

    # compute the dynamic emissivity for Landsat
    EMd = image.expression('fvc*0.99+(1-fvc)*em_bare',
      {'fvc':image.select('FVC'),'em_bare':emiss_bare})

    # compute emissivity directly from ASTER
    # without vegetation correction
    # get ASTER emissivity
    aster = ee.Image("NASA/ASTER_GED/AG100_003") \
      .clip(image.geometry())
    EM0 = image.expression('c13*EM13 + c14*EM14 + c',{
      'EM13':aster.select('emissivity_band13').multiply(0.001),
      'EM14':aster.select('emissivity_band14').multiply(0.001),
      'c13':ee.Image(c13),
      'c14':ee.Image(c14),
      'c':ee.Image(c)
      })

    # select which emissivity to output based on user selection
    EM = ee.Image(ee.Algorithms.If(use_ndvi,EMd,EM0))

    return image.addBands(EM.rename('EM'))

  return wrap

In [58]:
def get_lookup_table(fc, prop_1, prop_2):
  reducer = ee.Reducer.toList().repeat(2)
  lookup = fc.reduceColumns(reducer, [prop_1, prop_2])
  return ee.List(lookup.get('list'))

In [59]:
def LSTaddBand(landsat):

  def wrap(image):

    # coefficients for the Statistical Mono-Window Algorithm
    coeff_SMW_L8 = ee.FeatureCollection([
    ee.Feature(None, {'TPWpos': 0, 'A': 0.9751, 'B': -205.8929, 'C': 212.7173}),
    ee.Feature(None, {'TPWpos': 1, 'A': 1.0090, 'B': -232.2750, 'C': 230.5698}),
    ee.Feature(None, {'TPWpos': 2, 'A': 1.0541, 'B': -253.1943, 'C': 238.9548}),
    ee.Feature(None, {'TPWpos': 3, 'A': 1.1282, 'B': -279.4212, 'C': 244.0772}),
    ee.Feature(None, {'TPWpos': 4, 'A': 1.1987, 'B': -307.4497, 'C': 251.8341}),
    ee.Feature(None, {'TPWpos': 5, 'A': 1.3205, 'B': -348.0228, 'C': 257.2740}),
    ee.Feature(None, {'TPWpos': 6, 'A': 1.4540, 'B': -393.1718, 'C': 263.5599}),
    ee.Feature(None, {'TPWpos': 7, 'A': 1.6350, 'B': -451.0790, 'C': 268.9405}),
    ee.Feature(None, {'TPWpos': 8, 'A': 1.5468, 'B': -429.5095, 'C': 275.0895}),
    ee.Feature(None, {'TPWpos': 9, 'A': 1.9403, 'B': -547.2681, 'C': 277.9953})
    ])

    # Select algorithm coefficients
    coeff_SMW = ee.FeatureCollection(coeff_SMW_L8)

    # Create lookups for the algorithm coefficients
    A_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'A')
    B_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'B')
    C_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'C')

    # Map coefficients to the image using the TPW bin position
    A_img = image.remap(A_lookup.get(0), A_lookup.get(1),0.0,'TPWpos').resample('bilinear')
    B_img = image.remap(B_lookup.get(0), B_lookup.get(1),0.0,'TPWpos').resample('bilinear')
    C_img = image.remap(C_lookup.get(0), C_lookup.get(1),0.0,'TPWpos').resample('bilinear')

    # select TIR band
    tir = ee.String(ee.Algorithms.If(landsat == 'L8','B10',
                        ee.Algorithms.If(landsat == 'L7','B6_VCID_1',
                        'B6')))
    # compute the LST
    lst = image.expression(
      'A*Tb1/em1 + B/em1 + C',
         {'A': A_img,
          'B': B_img,
          'C': C_img,
          'em1': image.select('EM'),
          'Tb1': image.select(tir)
         }).updateMask(image.select('TPW').lt(0).Not())


    return image.addBands(lst.rename('LST'))
  
  return wrap

In [60]:
# cloudmask for TOA data
def cloudmasktoa(image):
  qa = image.select('BQA')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask)

In [61]:
# cloudmask for SR data
def cloudmasksr(image):
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(1 << 3) \
    .Or(qa.bitwiseAnd(1 << 5))
  return image.updateMask(mask.Not())

In [62]:
def LSTcollection(landsat, date_start, date_end, geometry, image_limit, use_ndvi):

  # load TOA Radiance/Reflectance
  collection_dict = ee.Dictionary(COLLECTION.get(landsat))

  landsatTOA = ee.ImageCollection(collection_dict.get('TOA')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmasktoa)
                #.limit(image_limit,'CLOUD_COVER_LAND') \
    
  # load Surface Reflectance collection for NDVI
  landsatSR = ee.ImageCollection(collection_dict.get('SR')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmasksr) \
                .map(NDVIaddBand(landsat)) \
                .map(FVCaddBand(landsat)) \
                .map(NCEP_TPWaddBand) \
                .map(EMaddBand(landsat,use_ndvi))
                #.limit(image_limit,'CLOUD_COVER_LAND') \

# combine collections
# all channels from surface reflectance collection
# except tir channels: from TOA collection
# select TIR bands
  tir = ee.List(collection_dict.get('TIR'))
  landsatALL = (landsatSR.combine(landsatTOA.select(tir), True))

  # compute the LST
  landsatLST = landsatALL.map(LSTaddBand(landsat))

  return landsatLST

# Compute LST mean

In [63]:
# const
start_date = '2015-01-01'
end_date = '2022-06-28'
landsat = 'L8' 
image_limit = 100
month_start = 1
month_end = 12
use_ndvi = False

In [64]:
# get list of urbanshift cities
georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/cities4forests/data/boundaries/geo_ref.csv') 
print(georef.to_string())

       city_name     city_boundary_name country_code
0       San_Jose      CRI-San_Jose-ADM2          CRI
1       Salvador      BRA-Salvador-ADM4          BRA
2         Bukavu        COD-Bukavu-ADM2          COD
3          Uvira         COD-Uvira-ADM3          COD
4    Brazzaville   COG-Brazzaville-ADM4          COG
5   Barranquilla  COL-Barranquilla-ADM4          COL
6    Addis_Ababa   ETH-Addis_Ababa-ADM4          ETH
7      Dire_Dawa     ETH-Dire_Dawa-ADM3          ETH
8        Nairobi       KEN-Nairobi-ADM3          KEN
9   Antananarivo  MDG-Antananarivo-ADM4          MDG
10   Mexico_City   MEX-Mexico_City-ADM2          MEX
11     Monterrey     MEX-Monterrey-ADM2          MEX


In [69]:
i = 1
boundary_id = georef.loc[i, 'city_boundary_name']
print(boundary_id)
boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/cities4forests/data/boundaries/'+boundary_id+'.geojson'

In [73]:
geo_name = georef.loc[i, 'country_code']+"-" + georef.loc[i, 'city_name']

'BRA-Salvador'

In [70]:
DistrictsGJ = requests.get(boundary_path).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
roi = Districts
ROIcenter = roi.geometry().centroid(1)

In [71]:
# get hottest period
Hottest_period = get_hottest_period(Districts = Districts,
                                    start_date = '2015-01-01',
                                    end_date = '2022-06-28',
                                   ROIcenter = ROIcenter)
date_start = Hottest_period[0]
date_end = Hottest_period[1]

2019-12-02
2020-02-29


In [72]:
# compute LST mean
LandsatColl = LSTcollection(landsat, date_start, date_end, roi, image_limit, use_ndvi).filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
LSTmean = LandsatColl.select('LST').reduce(ee.Reducer.mean()).subtract(273.15)

In [75]:
# store output locally
out_dir = os.getcwd()
city_file = 'data\\' + geo_name +'_LST_mean.tif'
filename = os.path.join(out_dir, city_file)

geemap.ee_export_image(
    LSTmean, 
    filename=filename, 
    scale=300, 
    region=roi.geometry(), 
    file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities4forests\scripts\data\BRA-Salvador_LST_mean.tif


In [ ]:
out_dir = os.getcwd()

In [78]:
for i in range(0,len(georef)):
    print(i)
    boundary_id = georef.loc[i, 'city_boundary_name']
    geo_name = georef.loc[i, 'country_code']+"-" + georef.loc[i, 'city_name']
    print(geo_name)
    
    # Get geometry
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/cities4forests/data/boundaries/'+boundary_id+'.geojson'
    DistrictsGJ = requests.get(boundary_path).json()
    Districts = geemap.geojson_to_ee(DistrictsGJ)
    roi = Districts
    ROIcenter = roi.geometry().centroid(1)
    
    # get hottest period
    Hottest_period = get_hottest_period(Districts = Districts,
                                        start_date = '2015-01-01',
                                        end_date = '2022-06-28',
                                       ROIcenter = ROIcenter)
    date_start = Hottest_period[0]
    date_end = Hottest_period[1]
    
    # compute LST mean
    LandsatColl = LSTcollection(landsat, date_start, date_end, roi, image_limit, use_ndvi).filter(ee.Filter.calendarRange(month_start, month_end, 'month'))
    LSTmean = LandsatColl.select('LST').reduce(ee.Reducer.mean()).subtract(273.15)
    
    # store locally
    city_file = 'data\\' + geo_name +'_LST_mean.tif'
    filename = os.path.join(out_dir, city_file)

    geemap.ee_export_image(
        LSTmean, 
        filename=filename, 
        scale=1000, 
        region=roi.geometry(), 
        file_per_band=False
    )

0
CRI-San_Jose
2017-02-04
2017-05-04
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities4forests\scripts\data\CRI-San_Jose_LST_mean.tif
1
BRA-Salvador
2019-12-02
2020-02-29
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities4forests\scripts\data\BRA-Salvador_LST_mean.tif
2
COD-Bukavu
2021-08-29
2021-11-26
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities4forests\scripts\data\COD-Bukavu_LST_mean.tif
3
COD-Uvira
2021-08-29
2021-11-26
Generating URL ...
Please wait ...
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\cities4forests\\scripts\\data\\COD-Uvira_LST_mean.zip'
4
COG-Brazzaville
2021-04-28
2021-07-26
Generating URL ...
Please wait ..